In [1]:
import numpy as np
import pandas as pd

import nltk
import re


In [2]:
punkt_path = "../input/punkt/punkt/english.pickle"
stopwords_path = "../input/stopwords/stopwords/english"

In [3]:
data = pd.read_csv("../input/tennis-articles/tennis_articles.csv", encoding = "ISO-8859-1")

In [4]:
data.head()

,article_id,article_title,article_text,source
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP)  Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [5]:
df = data.drop(columns=["article_title"])


In [6]:
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP)  Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [7]:
df['article_text'][1]

"BASEL, Switzerland (AP) \x97 Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours 

**Preprocessing**

In [8]:
from nltk.tokenize import sent_tokenize

In [9]:
sentences = []
for article in df['article_text'] :
    sentences.append(sent_tokenize(article))

sentences = [y for x in sentences for y in x]


Word Embedding - GloVe (Global vectors for Word representation)

In [10]:
glove_file = "../input/glove6b100dtxt/glove.6B.100d.txt"

In [11]:
# Load GloVe embeddings file
def load_glove_embeddings(embeddings_file):
    embeddings = {}
    with open(embeddings_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Example usage
glove_embeddings = load_glove_embeddings(glove_file)

In [12]:
len(glove_embeddings)

400000

In [13]:
word = "suraj"
if word in glove_embeddings :
    vec = glove_embeddings[word]
#     print(vec)
print(vec)

[-0.81987   -0.36602    0.010225   0.30581   -0.40235    0.27468
 -0.13116    0.56296    0.27195   -0.078814   0.23822    1.1835
 -0.26922   -0.01555    0.33746   -0.61731   -0.32372    0.16697
  0.084303   1.0162    -0.61549   -0.27345    0.67037   -0.15805
  0.068873   0.94155   -0.56538   -0.11317   -0.64022   -0.80405
 -0.36348   -0.31551   -0.45235   -0.0067441 -0.55904   -0.073975
  0.37722    0.32483   -0.0032219 -0.22344   -0.22925    0.89231
  0.26478   -0.62967    0.1059     0.32755   -0.24942    0.23814
  0.11616    0.92896   -0.90635    0.54281   -0.36051   -0.30946
 -0.11045    0.056876  -0.28454   -0.067826  -0.57253   -0.19346
  0.40167   -0.0078061  0.11282    0.61507   -0.43525   -0.056226
 -0.061505   0.0036911 -0.017236   0.59829   -0.20246   -0.048012
  0.19081   -0.50655   -0.76171   -0.12845   -0.86558   -0.56953
  0.50687    0.10509   -0.50987   -0.64946   -0.01097   -0.45895
 -0.23019    0.59051   -0.65717    0.33747   -0.66772    0.29055
 -0.41147    0.062058  

In [14]:
clean_sent = pd.Series(sentences).str.replace("[^a-zA-z]", " ")

In [15]:
clean_sent = [sent.lower() for sent in clean_sent]         

In [16]:
print(clean_sent[0])

maria sharapova has basically no friends as tennis players on the wta tour.


**Importing stopwords using NLTK package**

In [17]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

***Importing stopwords using dataset***

In [18]:
with open(stopwords_path, 'r') as f:
    stopwords = f.read().splitlines()
print(len(stopwords))

153


In [19]:
def remove_stopwords(sentence):
    new_sen = " ".join([i for i in sentence if i not in stop_words])
    return new_sen

In [20]:
print(len(stop_words))

179


In [21]:
clean_sentences = [remove_stopwords(sentence.split()) for sentence in clean_sent]

In [22]:
clean_sentences[:5]

['maria sharapova basically friends tennis players wta tour.',
 "russian player problems openly speaking recent interview said: 'i really hide feelings much.",
 'think everyone knows job here.',
 "i'm courts i'm court playing, i'm competitor want beat every single person whether they're locker room across net.",
 "i'm one strike conversation weather know next minutes go try win tennis match."]

**Vector representation of the sentences**

In [23]:
word_embeddings = {}
f = open(glove_file, encoding = 'utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    word_embeddings[word] = coefs
f.close()

In [24]:
word_embeddings['hello'].shape

(100,)

In [25]:
sentence_vectors = []
for i in clean_sentences:
    if(len(i) != 0) :
        v = sum([word_embeddings.get(w,np.zeros((100,))) for w in i.split()])/ (len(i.split() ) + 0.001)
    else :
        v = np.zeros((100,))
    sentence_vectors.append(v)

Similarity matrix - **Cosine similarity matrix**

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
n = len(sentences)
similarity_matrix = np.zeros([n, n])

In [28]:
for i in range(n):
    for j in range(n):
        if i!=j :
            a = sentence_vectors[i].reshape(1,100)
            b = sentence_vectors[j].reshape(1,100)
            similarity_matrix[i][j] = cosine_similarity(a,b)[0,0]

In [29]:
print(similarity_matrix.shape)

(130, 130)


Similarity matrix -> Graph

In [30]:
import networkx as nx

nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

**Summarization**

In [31]:
ranked_sentences = sorted(( (scores[i],s) for i,s in enumerate(sentences)), reverse=True)

Extract top 10 sentences as summary

In [32]:
for i in range(10):
    print(ranked_sentences[i][1])

I was on a nice trajectorythen, Reid recalled.If I hadnt got sick, I think I could have started pushing towards the second week at the slams and then who knows. Duringa comeback attempt some five years later, Reid added Bernard Tomic and 2018 US Open Federer slayer John Millman to his list of career scalps.
Full effort Nick could live out his tennis like a (Tomas) Berdych or (Jo- Wilfried) Tsonga, consistently making second week,quarters, semis, finals of slams - and then hopefully more.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
I just felt like it really kind of changed where people were a little bit, definitely in the '90s, a lot more quiet, into themselves, and then it started to become better. Meanwhile, Federer is hoping he can improve his service game as he hunts his ninth Swiss Indoors title this week.
I felt like the best weeks that I had to get to know pla